## The Challenge

As a wealth manager you have a number of investors who are associated with multiple investment Mandates. Each Mandate may have one or more Accounts. These investors can be part of one or more Households, and a Household can contain one or more Investors. You would like to be able to see the holdings for an investor based on each Mandate & Household they are associated with. 

You also have a number of Branches and would like to see the assets managed by each Branch. Furthermore each branch has a number of Financial Advisors and you would like to be able to see the assets managed by each Financial Advisor.

## The Solution

1) Create your Instrument universe using a range of identifiers

2) Set up a scope for each branch to hold the accounts

3) Create a LUSID Portfolio for each account

4) Create a Property to hold the investor, mandate, and household details

5) Set the initial holdings of the portfolio

6) Load market data (prices)

7) Create a Portfolio Group for each household

8) Conduct a valuation against each household

9) Consider other ways of grouping & valuing client accounts

In [ ]:
# Import LUSID
import lusid.models as models
import lusid
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', client.metadata.get_lusid_versions().build_version)

![Scopes](img/paper-lusid.gif)

### Create your instrument universe using a range of identifiers

Before you can take on any holdings for your client accounts you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

*Run the cell below to import your instrument universe*

In [ ]:
equity_instruments = pd.read_csv('data/households-instruments-equities.csv')
equity_instruments.head(n=20)

Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [ ]:
# Initialise your batch upsert request
batch_upsert_request = {}

# Using your instrument universe create your batch request
for index, instrument in equity_instruments.iterrows():

    # Specify the columns of your identifiers
    identifier_columns = ['Figi', 'Ticker', 'ClientInternal']
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier] = models.InstrumentIdValue(
            value=instrument[identifier])
    
    # Build your request and add it to the dictionary
    batch_upsert_request[instrument['InstrumentName']] = models.InstrumentDefinition(
        name=instrument['InstrumentName'],
        identifiers=identifiers)
    
# Call LUSID to upsert your instrument defintions
instrument_response = client.instruments.upsert_instruments(
    requests=batch_upsert_request)

# Pretty print the response
prettyprint.instrument_response(instrument_response)

![Scopes](img/households-instrumentmaster.gif)

### Set up a scope to hold the accounts

Your wealth managment company has a number of different branches. Each of these branches will be allocated a Scope in LUSID for its accounts. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for the scope of two of your branches*

In [ ]:
scope_id = import_data.create_scope_id()
branch_broomfield_scope = 'branch_{}'.format(scope_id)
scope_id = import_data.create_scope_id()
branch_miami_scope = 'branch_{}'.format(scope_id)
prettyprint.heading('Broomfield Branch Scope', branch_broomfield_scope)
prettyprint.heading('Miami Branch Scope', branch_miami_scope)

![Scopes](img/households-branch.gif)

### Create a portfolio for each account that exists with the branch

Now that you have decided on the name for your scope, you can create the portfolios to represent the client accounts inside this scope. You will import the account details from a CSV file.

*Run the cell below to import your client's account details*

In [ ]:
accounts = pd.read_csv('data/households-accounts.csv')
accounts.head(n=10)

With the account details loaded you can now create your client portfolios.

Note that every portfolio can be referenced by a unique code. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolios in the cell below you are creating it with a 'created' date of 1052 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolios*

In [ ]:
# A mapping between branch codes and scopes in LUSID
branch_mapping = {
    "broomf_CO_USA": branch_broomfield_scope,
    "miami_FL_USA": branch_miami_scope
}

# Iterate over the client accounts
for index, account in accounts.iterrows():

    # Set the creation date of your client portfolio 
    portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)
    
    # Build your request to create your client portfolio
    request = models.CreateTransactionPortfolioRequest(
        display_name=account['account_name'],
        code=account['account_code'],
        base_currency=account['currency'],
        description=account['description'],
        created=portfolio_creation_date,
        corporate_action_source_id=None,
        accounting_method='AverageCost',
        sub_holding_keys=None,
        properties=None)

    # Call LUSID to create your client portfolio
    response = client.transaction_portfolios.create_portfolio(
        scope=branch_mapping[account['branch_id']],
        create_request=request)

    # Pretty print the response
    prettyprint.portfolio_response(response)

![Scopes](img/households-portfolio.gif)

### Create a property to hold the investor and the household

To keep track of the investor associated with each portfolio as well as the household you can make use of LUSID's extensible properties. These allow you to define a bespoke schema for your portfolio objects. Read more about properties in the [LUSID Knowledge Base: Properties](https://support.lusid.com/what-is-a-property). 

For further usage of the create property definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create a property to hold household_id, the account owner ids & the financial advisor id*

In [ ]:
# The property codes to create
property_codes = [
    'household_id', 
    'primary_acount_owner_id', 
    'other_account_owner_id',
    'financial_advisor_id',
    'mandate_id',
    'mandate_description']

# Iterate over the property codes
for property_code in property_codes:

    # Create your request to define a new property
    request = models.CreatePropertyDefinitionRequest(
        domain='Portfolio',
        scope=scope,
        code=property_code,
        value_required=False,
        display_name=property_code,
        data_type_id=models.ResourceId(scope='default', code='string'))

    # Call LUSID to create your new property
    response = client.property_definitions.create_property_definition(
        definition=request)

    # Grab the key off the response to use when referencing this property in other LUSID calls
    portfolio_property_key = response.key

    # Pretty print your key
    prettyprint.heading('Portfolio Property Key', portfolio_property_key)

With these properties created you can now populate them for each portfolio.

*Run the cell below to populate the values for these properties for each portfolio*

In [ ]:
# Iterate over each account
for index, account in accounts.iterrows():
    
    # Add the relevant account properties to the portfolio
    response = client.portfolios.upsert_portfolio_properties(
        scope=scope,
        code=account['account_code'],
        portfolio_properties={
            'Portfolio/{}/{}'.format(scope, account_property): models.PropertyValue(
                label_value=account[account_property]
            ) for account_property in property_codes
        }
    )

    # Pretty print the response 
    prettyprint.portfolio_properties_response(response)
    print ('\n')

![Scopes](img/households-properties.gif)

### Set the initial holdings of the portfolio

Now that you have your instrument universe populated and portfolios created you can load your current client holdings into their portfolios. In this case you will import their holdings from a CSV file. 

*Run the cell below to import your current client holdings*

In [ ]:
holdings = pd.read_csv('data/households-holdings.csv')
holdings.head(n=20)

Now that you have imported your client holdings you can add them to LUSID. You can do this by setting the holdings on a portfolio. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The effect of holding adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio).

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to upsert your holdings into LUSID*

In [ ]:
# Make the holdings effective from now
holdings_effective_date = datetime.now(pytz.UTC)

# Iterate the portfolios in the holdings CSV, note in this case you only have one
for portfolio in holdings['portfolio_code'].unique():
    
    # Initialise a list to hold your adjustments
    holding_adjustments = []
    
    # Iterate over the holdings in each portfolio
    for index, holding in holdings.loc[holdings['portfolio_code'] == portfolio].iterrows():
        
        # Set your instrument identifiers based on whether or not instrument is cash
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifer = holding['instrument_name'].split('_')[0]
        else:
            identifier_key = 'Instrument/default/Figi'
            identifer = holding['figi']
            
        # Create your holding adjustment and append it to your list
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifer},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=holding['currency']),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ]
            )
        )
    
    # Call LUSID to set your initial holdings
    response = client.transaction_portfolios.set_holdings(
        scope=scope,
        code=portfolio,
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)

    # Pretty print our response from LUSID
    prettyprint.set_holdings_response(response, scope, portfolio)

### Create a portfolio group for each household

Now that you've created Portfolios for the client accounts, you can group them together into households using Portfolio Groups.

Read more about portfolio groups here [LUSID Knowledge Base: How do you Group and Aggregate Portfolios?](https://support.lusid.com/how-do-you-group-and-aggregate-portfolios)

*Run the cell below to create a portfolio group for each household*

In [ ]:
# Iterate over all households
for household in accounts['household_id'].unique():
    
    # Build a create group request for this group
    group_request = models.CreatePortfolioGroupRequest(
    code=household+'-Group',
    display_name='Contains all accounts for the {} household'.format(household))

    # Call LUSID to create the portfolio group for the household
    response = client.portfolio_groups.create_portfolio_group(
        scope=scope,
        request=group_request)

    # Pretty print the response
    prettyprint.portfolio_group_response(response, 'created')

You can now add the portfolios for each household to the appropriate group.

*Run the cell below to add all of the portfolios in each household to the group*

In [ ]:
# Call LUSID to list all the portfolios for this branch
portfolios = client.portfolios.list_portfolios_for_scope(
    scope=scope) 

# Iterate over each portfolio
for portfolio in portfolios.values:
    
    # For this portfolio get its properties which you defined earlier
    properties = client.portfolios.get_portfolio_properties(
        scope=portfolio.id.scope, 
        code=portfolio.id.code)
    
    # Make the list of properties easy to work with by converting them to key-value pairs
    portfolio_properties = {prop.key: prop.value for prop in properties.properties}
    
    # Using the household_id property determine which portfolio group to add this portfolio too
    response = client.portfolio_groups.add_portfolio_to_group(
        scope=scope,
        code=portfolio_properties['Portfolio/{}/household_id'.format(scope)]+'-Group',
        portfolio_id=models.ResourceId(
            scope=portfolio.id.scope,
            code=portfolio.id.code))
    
    # Pretty print the response
    prettyprint.get_portfolio_group_response(response)

![Scopes](img/households-portfoliogroupshouseholds.gif)

### Load market data prices

With the Portfolio Groups created to group each household together, you can now aggregate across households.

To aggregate & value a portfolio in LUSID you need to upsert market data quotes against the underlying holdings or specify an analytics library to use. Read more about aggregating and valuing portfolios in the [LUSID Knowledge Base: Aggregations and Valuations](https://support.lusid.com/what-is-a-valuation).

In this case you will upsert market data quotes to the quote store to be used in an aggregation request. You will import these quotes from a CSV file.

*Run the cell below to import the market data prices*

In [ ]:
# Import the market data prices
prices = pd.read_csv('data/households-prices.csv')
prices.head(n=50)

Now that you have imported the market data you can add it to the quote store in LUSID. Read more about what a quote is in the [LUSID Knowledge Base: What is a Quote?](https://support.lusid.com/what-is-a-quote).

For further usage of the Upsert Quotes API call refer to the [LUSID API Docs: Upsert Quotes](https://docs.lusid.com/#operation/UpsertQuotes).

*Run the cell below to upsert the market data quotes into LUSID*

In [ ]:
# Initialise an empty list to hold the market data quotes
instrument_quotes = []

# Iterate over each quote
for index, quote in prices.iterrows():
    
    # Get the LUSID Instrument ID for the quoted instrument
    luid = client.search.instruments_search(
        symbols=[
            models.InstrumentSearchProperty(
                key='Instrument/default/Figi',
                value=quote['figi'])
        ],
        mastered_only=True
        )[0].mastered_instruments[0].identifiers['LusidInstrumentId'].value
    
    # Create a quote for this instrument and append it to the list of quotes
    instrument_quotes.append(models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            provider='DataScope',
            price_source='',
            instrument_id=luid,
            instrument_id_type='LusidInstrumentId',
            quote_type='Price',
            price_side='Mid'),
        metric_value=models.MetricValue(
            value=quote['price_current'],
            unit=quote['currency']),
        effective_at=holdings_effective_date,
        lineage='InternalSystem'
    ))

# Upsert the quotes into LUSID
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=instrument_quotes)

# Pretty print the response
prettyprint.upsert_quotes_response(response)

### Conduct a valuation against each household

Now that the quotes have been added to the quote store you can aggregate across each household. The logic for an aggregation is controled by a LUSID recipe. Read more about recipes in the [LUSID Knowledge Base: What is a Recipe and How Are They Used?](https://support.lusid.com/what-is-a-recipe-and-how-are-they-used).

For further usage of the Get Aggregation by Portfolio API call refer to the [LUSID API Docs: Get Aggregation by Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio).

*Run the cell below to aggregate and value each household's investments*

In [ ]:
# Create an inline recipe to configue the aggregation logic
inline_recipe = models.ConfigurationRecipe(
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.LusidInstrumentId.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='LusidInstrumentId',
            default_scope=scope)
        )
    )

aggregation_results_df = []

# Iterate over each household
for household in accounts['household_id'].unique():
    
    # Create the aggregation request
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe,
        effective_at=holdings_effective_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='Sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='Sum'),
            models.AggregateSpec(key='Holding/default/Units',
            op='Sum'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value')
        ],
        group_by=[
            'Instrument/default/Name'
        ])

    # Call LUSID to perform an aggregation
    response = client.aggregation.get_aggregation_by_group(
        scope=scope,
        code=household+'-Group',
        request=aggregation_request)

    # Pretty print the response
    aggregation_results_df.append(prettyprint.aggregation_response_generic_df(
        response=response, 
        index_key="Instrument/default/Name", 
        name=household))

In [ ]:
prettyprint.heading('Household: ', aggregation_results_df[0].name)
aggregation_results_df[0]

In [ ]:
prettyprint.heading('Household: ', aggregation_results_df[1].name)
aggregation_results_df[1]

Perhaps you might also like to group your client accounts by financial advisor. Using Portfolio Groups you can do this as well.

*Run the cell below to create a Portfolio Group for each advisor*

In [ ]:
# Iterate over all financial advisors
for advisor in accounts['financial_advisor_id'].unique():
    
    # Build a create group request for this advisor
    group_request = models.CreatePortfolioGroupRequest(
    code=advisor+'-Group',
    display_name='Contains all accounts for the financial advisor with id {}'.format(advisor))

    # Call LUSID to create the portfolio group for the advisor
    response = client.portfolio_groups.create_portfolio_group(
        scope=scope,
        request=group_request)

    # Pretty print the response
    prettyprint.portfolio_group_response(response, 'created')

You can now add the portfolios for each advisor to the appropriate group.

*Run the cell below to add all of the portfolios in for each advisor to the appropriate group*

In [ ]:
# Call LUSID to list all the portfolios for this branch
portfolios = client.portfolios.list_portfolios_for_scope(
    scope=scope) 

# Iterate over each portfolio
for portfolio in portfolios.values:
    
    # For this portfolio get its properties which you defined earlier
    properties = client.portfolios.get_portfolio_properties(
        scope=portfolio.id.scope, 
        code=portfolio.id.code)
    
    # Make the list of properties easy to work with by converting them to key-value pairs
    portfolio_properties = {prop.key: prop.value for prop in properties.properties}
    
    # Using the financial_advisor_id property determine which portfolio group 
    # to add this portfolio too
    response = client.portfolio_groups.add_portfolio_to_group(
        scope=scope,
        code=portfolio_properties['Portfolio/{}/financial_advisor_id'.format(scope)]+'-Group',
        portfolio_id=models.ResourceId(
            scope=portfolio.id.scope,
            code=portfolio.id.code))
    
    # Pretty print the response
    prettyprint.get_portfolio_group_response(response)

![Scopes](img/households-portfoliogroupsadvisor.gif)

Now you can aggregate by financial advisor.

*Run the cell below to aggregate your client accounts by financial advisor*

In [ ]:
aggregation_results_df = []

# Iterate over each household
for advisor in accounts['financial_advisor_id'].unique():
    
    # Create the aggregation request
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe,
        effective_at=holdings_effective_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='Sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='Sum'),
            models.AggregateSpec(key='Holding/default/Units',
            op='Sum'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value')
        ],
        group_by=[
            'Instrument/default/Name'
        ])

    # Call LUSID to perform an aggregation
    response = client.aggregation.get_aggregation_by_group(
        scope=scope,
        code=advisor+'-Group',
        request=aggregation_request)

    # Append the response to the list of responses
    aggregation_results_df.append(prettyprint.aggregation_response_generic_df(
        response=response, 
        index_key="Instrument/default/Name", 
        name=advisor))

In [ ]:
prettyprint.heading('Financial Advisor: ', aggregation_results_df[0].name)
aggregation_results_df[0]

In [ ]:
prettyprint.heading('Financial Advisor: ', aggregation_results_df[1].name)
aggregation_results_df[1]

You can also create a portfolio group to see the investment across all the client accounts.

*Run the cell below to create a portfolio group for all accounts*

In [ ]:
# Build a create group request for this group
group_request = models.CreatePortfolioGroupRequest(
code='AllAccounts',
display_name='Contains all client accounts for {}'.format(scope))

# Call LUSID to create the portfolio group for all client accounts
response = client.portfolio_groups.create_portfolio_group(
    scope=scope,
    request=group_request)

# Pretty print the response
prettyprint.portfolio_group_response(response, 'created')

In [ ]:
# Call LUSID to list all the portfolios for this branch
portfolios = client.portfolios.list_portfolios_for_scope(
    scope=scope) 

# Iterate over each portfolio
for portfolio in portfolios.values:
    
    # Add the portfolio the all accounts portfolio group
    response = client.portfolio_groups.add_portfolio_to_group(
        scope=scope,
        code='AllAccounts',
        portfolio_id=models.ResourceId(
            scope=portfolio.id.scope,
            code=portfolio.id.code))
    
    # Pretty print the response
    prettyprint.get_portfolio_group_response(response)

![Scopes](img/households-portfoliogroupsallaccounts.gif)

You can now aggregate across all client accounts.

*Run the cell below to aggregate across all client accounts*

In [ ]:
aggregation_results_df = []

# Create the aggregation request
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/SubHoldingKey',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/Units',
        op='Sum'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value')
    ],
    group_by=[
        'Instrument/default/Name'
    ])

# Call LUSID to perform an aggregation
response = client.aggregation.get_aggregation_by_group(
    scope=scope,
    code='AllAccounts',
    request=aggregation_request)

# Add the response to the list
aggregation_results_df.append(prettyprint.aggregation_response_generic_df(
        response=response, 
        index_key="Instrument/default/Name", 
        name=scope))

In [ ]:
prettyprint.heading('Branch: ', aggregation_results_df[0].name)
aggregation_results_df[0]